In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import regexp_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
!python -m spacy download es_core_news_sm
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from collections  import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
You should consider upgrading via the 'c:\Users\Usuario\Desktop\lautaro\ies 21\IES21\4to cuatrimestre\PLN\entorno\Scripts\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 12.9/12.9 MB 4.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [5]:
df= pd.read_excel(r'C:\Users\Usuario\Desktop\lautaro\ies 21\IES21\4to cuatrimestre\PLN\Noticias Verdaderas y Falsas.xlsx')
df

,class,Text
0,True,Algunas de las voces extremistas más conocida...
1,True,Después de casi dos años y medio de luchas po...
2,True,Dos periodistas birmanos de la agencia Reuters...
3,True,El Cuerpo Nacional de Policía ha detenido a cu...
4,True,El desfile de la firma en Roma se convierte en...
...,...,...
1995,True,El Consejo de Gobierno ha dado su visto bueno...
1996,True,Investigadores valencianos han desarrollado u...
1997,True,Los arrestados actuaban en coches y en establ...
1998,True,El Rey ha encargado este miércoles a Pedro Sá...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   2000 non-null   bool  
 1   Text    2000 non-null   object
dtypes: bool(1), object(1)
memory usage: 17.7+ KB


In [7]:
def preprocesamiento(documento):
# Eliminación de valores numéricos
  documento = re.sub('\d', ' ', documento)
  documento = documento.lower()
############################## TOKENIZAR ##############################
# Tokenizo con una List Comprehension, y luego a esta le filtro las stopwords
  tokenizado = [token for token in regexp_tokenize(documento, pattern='\w+')]
  stops= set(stopwords.words('spanish'))
  filtro = [filtro for filtro in tokenizado if filtro not in stops]
############################## LEMATIZAR ##############################
# Spacy tiene su propio tokenizador 
  nlp= spacy.load('es_core_news_sm')
  doc= nlp(documento)
# Lematizo al texto, para eliminár los signos de puntuación o demas signos que no me aportarán al análisis, se filtrarán los tokens
# que están en la anterior List Comprehension llamada filtro
  lemma = [lema.lemma_ for lema in doc if str(lema) in filtro]
  return lemma

In [ ]:
docu= 'El congresista había adelantado que algo iba a pasar en la mañana de este miércoles, que aparentemente lo perjudicaría. “El peor fuego es el fuego ‘amigo’. Se vienen cosas oscuras y graves. ¡Ampliaremos!”, escribió en su cuenta de Twitter. Una vez se conoció la decisión del tribunal, añadió'
pre= preprocesamiento(docu)
pre

In [8]:
pre_procesamiento=  [preprocesamiento(documento) for documento in df['Text']]

KeyboardInterrupt: 

#### palabras mas frecuentes

In [80]:
from itertools import chain
lista= list(chain( * pre_procesamiento ))

In [ ]:
x=Counter(lista).most_common()
x

In [ ]:
dict_1=dict()
for key,values in x: 
    dict_1.setdefault(key, values)
dict_1 

In [105]:
df = pd.DataFrame([[key, dict_1[key]] for key in dict_1.keys()], columns=['Name', 'Cant'])
grupo=df.groupby(['Cant'])['Cant'].count().reset_index(name='cantidad')
grupo

,Cant,cantidad
0,1,5305
1,2,1494
2,3,689
3,4,463
4,5,297
...,...,...
93,177,1
94,191,1
95,235,1
96,257,1


In [102]:
df

,Name,Cant
0,año,318
1,poder,257
2,ser,235
3,según,191
4,gobierno,177
...,...,...
9766,microbiología,1
9767,cnm,1
9768,barakaldo,1
9769,rabia,1


In [107]:
import plotly.express as px
fig = px.bar(df, x='Name', y='Cant')
fig.show()

#### continua

In [ ]:
Df_Tf_IDF= pd.DataFrame()
vectorizer = TfidfVectorizer()
for palabra in df_lema['lema']:
  vectores = vectorizer.fit_transform([str(palabra)])
  nombres= vectorizer.get_feature_names()
  dense= vectores.todense()
  dense_list= dense.tolist()
  Df_Tf_IDF=Df_Tf_IDF.append(dense_list,columns=nombres)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectores = vectorizer.fit_transform([str(filtro_rugby),str(filtro_mina),str(filtro_politica),str(filtro_economia)])
nombres= vectorizer.get_feature_names()
dense= vectores.todense()
dense_list= dense.tolist()
df= pd.DataFrame(dense_list,columns=nombres )
df